In [3]:
import asyncio
import chainlit as cl

from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage, AIMessage
from similarity import similarity_search
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langchain_openai import ChatOpenAI
# Initialize LLM

llm = ChatOpenAI(model="gpt-5-mini")

# Initialize Tavily search client
tavily = TavilySearch(max_results=2)

@tool
def google_search(query: str) -> str:
    """it is used to do google search"""
    results = tavily.run(query)
    return results

@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["bangalore"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "bangalore" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "bangalore" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."
        
tools = [get_weather, check_seating_availability, google_search]

llm_with_tools = llm.bind_tools(tools)

result = llm_with_tools.invoke("How will the weather be in bangalore today?")
print(result)

2025-10-02 15:08:37 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
content='' additional_kwargs={'tool_calls': [{'id': 'call_RLrrW39MStnOpeF6qy2j0NKH', 'function': {'arguments': '{"location":"Bengaluru, India"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 185, 'total_tokens': 213, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMAWiq5ek07N4pD5MwYSssEDfnEqE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--40ffdf8f-63d0-4fe4-94ee-a4babc971eb9-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Bengaluru, India'}, 'id': 'call_RLrrW39MStnOpeF6qy2j0NKH', 'type': 'tool_call'}] u

In [4]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Bengaluru, India'},
  'id': 'call_RLrrW39MStnOpeF6qy2j0NKH',
  'type': 'tool_call'}]

In [5]:
result = llm_with_tools.invoke(
    "How will the weather be in bangalore today? Do you still have seats outdoor available?"
)

2025-10-02 15:08:45 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [6]:
 result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_B8S74AwEbZdVrPKlrO4yMzrz', 'function': {'arguments': '{"location": "Bangalore, India"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_TPm3V9TY6ouTI2kiuT9dNPeN', 'function': {'arguments': '{"location": "Bangalore", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 193, 'total_tokens': 324, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMAWnGD3riZDky8epjNVLb2BuB52w', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--76f9c81f-ca6a-4fa7-a9ff-f134953d165e-0', tool_calls=[{'name': 'get_weather', 'args':

In [7]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Bangalore, India'},
  'id': 'call_B8S74AwEbZdVrPKlrO4yMzrz',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Bangalore', 'seating_type': 'outdoor'},
  'id': 'call_TPm3V9TY6ouTI2kiuT9dNPeN',
  'type': 'tool_call'}]

# Implement Tool Calling

In [8]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability
}

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

In [10]:
messages = [
    HumanMessage(
        "How will the weather be in bangalore today? Do you still have seats outdoor available?"
    )
]

In [11]:
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

2025-10-02 15:12:44 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [12]:
messages

[HumanMessage(content='How will the weather be in bangalore today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Rmajv9EpyiTJSoQTCUItxstu', 'function': {'arguments': '{"location": "Bangalore"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_TYZlGuoPyw3jgy2lK8J3kXXU', 'function': {'arguments': '{"location": "Bangalore", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 193, 'prompt_tokens': 193, 'total_tokens': 386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMAafEAmi2f6CzrHcVN3jn86x8NG8', 'service_tier': 'd

In [13]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [14]:
messages

[HumanMessage(content='How will the weather be in bangalore today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Rmajv9EpyiTJSoQTCUItxstu', 'function': {'arguments': '{"location": "Bangalore"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_TYZlGuoPyw3jgy2lK8J3kXXU', 'function': {'arguments': '{"location": "Bangalore", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 193, 'prompt_tokens': 193, 'total_tokens': 386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMAafEAmi2f6CzrHcVN3jn86x8NG8', 'service_tier': 'd

In [15]:
res = llm_with_tools.invoke(messages)
res

2025-10-02 15:16:17 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Today in Bangalore it’s about 15°C and cloudy. Yes — we still have outdoor seats available.\n\nWould you like me to reserve a table? If so, tell me the date/time and party size (and any special requests). Note: weather can change, so we can move you to covered seating if needed.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 265, 'prompt_tokens': 285, 'total_tokens': 550, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMAe3DOPT4CYCbp6Ph8utheIrG6DX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8a4a3fab-044b-431f-8c78-a1e5a9ec3e05-0', usage_metadata={'input_tokens': 285, 'output_tokens': 265, 'total_tokens': 550, 'input_token_details': {'audio': 0

In [16]:
print(res.content)

Today in Bangalore it’s about 15°C and cloudy. Yes — we still have outdoor seats available.

Would you like me to reserve a table? If so, tell me the date/time and party size (and any special requests). Note: weather can change, so we can move you to covered seating if needed.
